# 🏗️ Notebook: Einführung in Retrieval-Augmented Generation (RAG)

In diesem Notebook lernen wir, wie wir **Retrieval-Augmented Generation (RAG)** nutzen können, um die Antworten von großen Sprachmodellen (LLMs) durch externe Wissensquellen zu verbessern. Wir werden eine einfache Anwendung erstellen, die Fragen zu einem gegebenen Text beantwortet.

## 📚 Quellen

- [LangChain Dokumentation zu RAG](https://python.langchain.com/docs/tutorials/rag/)


---

Viel Erfolg beim Ausprobieren und Validieren! 🤗

In [ ]:
%%capture
# Installieren wir zunächst einige Packete die uns LangChain zur Verfügung stellen. U.a. eine Schnittstelle zu Ollama.
!pip install langchain langchain-community langchain-ollama rank-bm25 chromadb ipywidgets

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
LLM_URL = "http://132.199.138.16:11434"
LLM_MODEL = "gemma3:12b"
EMBEDDING_MODEL = "nomic-embed-text" # Unser Ollama-Server bei der Uni stellt uns ein Embedding Modell zur Verfügung

In [ ]:
# LLM initialisieren
llm = ChatOllama(
    model=LLM_MODEL,
    base_url=LLM_URL,
    temperature=0
)

## 1. Dokumente laden und vorbereiten

Zunächst laden wir unser Wissens-Dokument und teilen es in kleinere Chunks auf. Diese Chunks sind die Basis für die spätere Suche.

Bei dem Dokument handelt es sich um eine Publikation von unserem Lehrstuhl.

In [ ]:
# Dokument laden
# TextLoader ist eine LangChain-Klasse zum Einlesen von Textdateien
# Sie liest die gesamte Datei ein und erstellt daraus ein Langchain Document-Objekt (https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html).
# encoding="utf-8" stellt sicher, dass Umlaute und Sonderzeichen korrekt gelesen werden
loader = TextLoader("dummy_wissen.txt", encoding="utf-8")
document = loader.load()  # load() gibt eine Liste von Document-Objekten zurück, in diesem Fall mit nur einem Element

# Text in Chunks aufteilen
# RecursiveCharacterTextSplitter teilt lange Texte in kleinere Abschnitte (Chunks)
# Dies ist wichtig für RAG-Systeme, da Embedding-Modelle oft Längenbeschränkungen haben von z:.B. 512 oder 1024 Token
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Maximale Länge eines Chunks in Zeichen
    chunk_overlap=50,      # Überlappung zwischen Chunks verhindert Informationsverlust an Grenzen
    separators=["\n", ",", " ", ""]       # Text wird primär an Absätzen (Doppel-Zeilenumbrüchen) getrennt
)
# split_documents() wendet das Splitting auf das Document-Objekt an
# und gibt eine Liste von kleineren Document-Objekten zurück
docs = text_splitter.split_documents(document)

# page_content enthält den eigentlichen Text eines Document-Objekts
print(f"Anzahl der Chunks: {len(docs)}")
print(f"\nBeispiel-Chunk:\n{docs[0].page_content}")

Der `RecursiveCharacterTextSplitter` versucht die Separatoren von oben nach unten in der angegebenen Reihenfolge. Er nimmt immer den ersten Separator, der den Text in Chunks `≤` `chunk_size` aufteilen kann. Falls ein Chunk trotzdem noch zu groß ist, probiert er den nächsten Separator in der Liste, bis der Text klein genug ist.

Der `chunk_overlap` sorgt dafür, dass aufeinanderfolgende Chunks sich überlappen. Die letzten X Zeichen eines Chunks werden am Anfang des nächsten Chunks wiederholt, damit wichtige Informationen, die über Chunk-Grenzen hinausgehen, nicht verloren gehen und der semantische Kontext erhalten bleibt. Mehr zu dem Thema Text-Splitting [hier](https://dev.to/tak089/what-is-chunk-size-and-chunk-overlap-1hlj#:~:text=Improves%20Retrieval%3A%20Overlapping%20chunk%20helps,smaller%20pieces%2C%20allowing%20efficient%20processing.)

```python

```


## 2. RAG mit Keyword-basierter Suche (BM25)

BM25 ist ein klassischer Ranking-Algorithmus, der auf **Keyword-Matching** basiert. Er berechnet, wie relevant ein Dokument für eine Suchanfrage ist, basierend auf der Häufigkeit und Verteilung der Suchbegriffe.

In [ ]:
# BM25 Retriever erstellen
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2  # Anzahl der zurückgegebenen Dokumente

# Beispiel-Suche mit dem BM25-Retriever
# mit .invoke() werden die 2 relevantesten Dokumente zurückgegeben
query = "Wann beginnt das Wintersemester?"
retrieved_docs = bm25_retriever.invoke(query)

# print(f"Query: {query}\n")
# print("Gefundene Dokumente:")
# for i, doc in enumerate(retrieved_docs):
#     print(f"\n--- Dokument {i+1} ---")
#     print(doc.page_content)

In [ ]:
# Erstellen wir eine promot mit PromptTemplate
# Das Template enthält Platzhalter für Kontext und Frage
# Diese werden später durch die tatsächlichen Werte ersetzt
template = """Beantworte die Frage basierend auf dem folgenden Kontext:

Kontext: {context}

Frage: {question}

Antwort:"""

# Die input_variables müssen mit den Platzhaltern im Template übereinstimmen
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain_bm25 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=bm25_retriever,
    chain_type_kwargs={"prompt": prompt}
)

# Frage stellen
question = "Wer neben Christian Wolff hat das Paper geschrieben?"
result = qa_chain_bm25.invoke({"query": question})
print(f"Frage: {question}")
print(f"Antwort: {result['result']}")

## 3. RAG mit Embeddings (Semantische Suche)

Bei der **Embedding-basierten Suche** werden Texte in numerische Vektoren umgewandelt, die ihre semantische Bedeutung repräsentieren. 

Wir verwenden ChromaDB (oft als "Chroma" bezeichnet), eine Open-Source Vector Datenbank. ChromaDB speichert Veektore effizient und ermöglicht schnelle Ähnlichkeitssuchen.

Mit `vectorstore.get()` könnten wir alle Vektoren abrufen als Python-Dictionary.

In [ ]:
from langchain_community.vectorstores import Chroma

# Embeddings-Modell initialisieren
ollama_embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=LLM_URL
)

# Mit Chroma einen Vektor-Speicher aus den Dokumenten erstellen
vectorstore = Chroma.from_documents(docs, ollama_embeddings)

# Retriever erstellen. Der Parameter k gibt an, wie viele ähnliche Dokumente zurückgegeben werden sollen
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Beispiel-Suche
query = "Was ist die E-Mail Addresse von Udo Kruschwitz?"
retrieved_docs = vector_retriever.invoke(query)

print(f"Query: {query}\n")
print("Gefundene Dokumente:")
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Dokument {i+1} ---")
    print(doc.page_content)

In [ ]:
# RAG-Chain mit Embeddings erstellen
qa_chain_embeddings = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_retriever,
    chain_type_kwargs={"prompt": prompt}
)

# Frage stellen
result = qa_chain_embeddings.invoke({"query": query})
print(f"Frage: {query}")
print(f"Antwort: {result['result']}")

### Übungsaufgabe: Daten aus CSV

Implementiere ein Retrieval-Augmented Generation (RAG) System, das Fragen zu den YouTube Top 100 Songs 2025 beantwortet.

Wir können nach folgenden Schritten vorgehen:

1. Daten laden: Lade die CSV-Datei `youtube-top-100-songs-2025.csv` mit dem CSVLoader
2. Chunking: Teile die Dokumente in Chunks (max. 500 Zeichen, 50 Zeichen Überlappung)
3. Embeddings & Vector Store: Erstelle Embeddings mit Ollama und speichere sie in einer ChromaDB
4. Retriever: Konfiguriere einen Retriever, der die 5 relevantesten Chunks findet
5. RAG-Chain: Erstelle eine RetrievalQA-Chain mit einem Custom-Prompt-Template
6. Testen: Beantworte z.B. die Frage: "Wie lange ist das Musikvideo zu 'ALL MY LOVE' von Coldplay?"

In [ ]:
# Hier kann der Code geschrieben werden...

<details>
<summary><b>Lösung anzeigen</b></summary>

```python
from langchain.document_loaders import CSVLoader

# Laden wir nun mit CSVLoader die YouTube Top 100 Songs 2025 CSV-Datei
# Hinweis: Oft kann es Sinn machen, die CSV-Datei noch vorzubereiten, z.B. irrelevante Spalten zu entfernen
# oder fehlende Werte zu ergänzen. Erinnerung: Dies kann z.B. mit Pandas gemacht werden.
loader = CSVLoader(
    file_path="youtube-top-100-songs-2025.csv", encoding="utf-8")
documents = loader.load()

# In Chunks aufteilen
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n", ",", " ", ""]
)
chunks = splitter.split_documents(documents)

# Embeddings erzeugen und temporäre Chroma-DB erstellen
ollama_embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=LLM_URL
)
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=ollama_embeddings
)

# Retriever definieren
vector_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5})  # 5 relevante Chunks abrufen

# Beispiel-Suche
query = "Wie lange ist das Musikvideo zu 'ALL MY LOVE' von Coldplay?"

# Relevante Dokumente abrufen
retrieved_docs = vector_retriever.invoke(query)

template = """Beantworte zu den top 100 YouTube Musikvideos die Frage basierend auf dem folgenden Kontext:

Kontext: {context}

Frage: {question}

Antwort:"""

# Die input_variables müssen mit den Platzhaltern im Template übereinstimmen
prompt = PromptTemplate(template=template, input_variables=[
                        "context", "question"])

# RAG-Chain mit Embeddings erstellen
qa_chain_embeddings = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_retriever,
    chain_type_kwargs={"prompt": prompt}
)

# Frage stellen
result = qa_chain_embeddings.invoke({"query": query})
print(f"Frage: {query}")
print(f"Antwort: {result['result']}")
```

</details>